In [1]:
import os 
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key="sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV")

# Load document
* txt/pdf/html/csv/markdown/json/directory

### load document using textloader

In [9]:
from langchain_community.document_loaders import TextLoader
from pathlib import Path

path = Path('~/OneDrive/바탕 화면/API 스터디/openai.txt').expanduser()

loader = TextLoader(path, encoding='utf-8')
docs = loader.load()

In [10]:
len(docs)

1

### load document using pypdfloader

In [48]:
from langchain_community.document_loaders import PyPDFLoader

# pdf 로드
loader = PyPDFLoader(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디\Chapter 7.pdf")
# 텍스트 추출
docs = loader.load()
docs

[Document(page_content='7\nDistance and Approximation\nCopyright ©  Cengage Learning. All rights reserved. ', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 0}),
 Document(page_content='Copyright ©  Cengage Learning. All rights reserved. \n7.4The Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 1}),
 Document(page_content='3\nThe Singular Values of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 2}),
 Document(page_content='4The Singular Values of a Matrix (1 of 1)\nDefinition\nIf A is an m ×n matrix, the singular values of A are the \nsquare roots of the eigenvalues of\n and are denoted by \nσ1, ... , σn. It is conventional to arrange the singular values so \nthat σ1≥ σ2≥ ... ≥ σn.', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 3}),
 Document(page_content='5Example 7.33\nFind the singula

### load documenets from directory

In [ ]:
# 디렉토리 안에 있는 파일을 자동으로 로드하고 처리함
from langchain_community.document_loaders import DirectoryLoader

# glob = * 는 모든 파일을 대상으로 로드한다는 뜻
loader = DirectoryLoader(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디", glob="*", show_progress=True)
docs = loader.load()

In [18]:
len(docs)

1083

# Chunking

### CharacterTextSplitter

In [3]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # 구분자 정의
    separator="\n",
    # 각 조각의 크기(500)
    chunk_size=500,
    chunk_overlap=100,
    # 파이썬 내장 함수 len 이용
    length_function=len,
    # separator를 문자열 그대로 처리
    is_separator_regex=False,
)

In [ ]:
documents = text_splitter.split_documents(docs)

In [ ]:
documents[0].page_content

### RecursiveCharacterSplitter
* 여러 개의 separator를 가지고 recursive하게 텍스트를 분리
* 첫 번째 separator로 분리 -> 너무 긴 청크는 다시 두 번째 separator로 분리..

In [22]:
# 텍스트를 재귀적으로 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # 점점 작은 단위로 쪼갬
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [23]:
documents = text_splitter.split_documents(docs)

 10%|█         | 1/10 [07:05<1:03:53, 425.95s/it]


In [24]:
for doc in documents:
    # 길이가 500 초과라면 출력함
    # chunk_size가 500이기 때문에 당연히 아무것도 출력 안됨
    if len(doc.page_content) > 500:
        print(doc)

In [25]:
documents[0]

Document(page_content='연세대학교 대학요람\n2022\n연세대학교\nYONSEI UNIVERSITY', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\대학 요람.pdf', 'page': 0})

### SemanticChunker
* OpenAI 임베딩 콜로 각 문장에 대한 embedding 생성
* next sentence와의 embedding distance가 threshold 이하일 때는 chunk를 결합하고, 이상일 때는 chunk 분리

In [ ]:
# 문서를 """의미 단위"""로 분할하는 클래스
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import OpenAIEmbeddings

# 객체 생성
text_splitter = SemanticChunker(OpenAIEmbeddings(api_key="sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV"))

# 입력 문서(docs)를 의미 단위로 분할함
documents = text_splitter.split_documents(docs)

In [ ]:
documents[0]

Document(page_content='OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work". As one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI spring.The organization consists of the non-profit OpenAI, Inc. registered in Delaware and its for-profit subsidiary OpenAI Global, LLC. It was founded by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk serving as the initial Board of Directors members. Microso

### CharacterTokenSplitter
* tiktoken(OPENAI가 개발한 byte pair tokenizer)를 이용해서 tokenize
* chunk size는 토큰의 개수
* LM의 컨텍스트 제한은 보통 토큰 사이즈로 정해지기 때문에 토큰 단위로 나누는 것은 합리적인 선택

In [50]:
# 텍스트를 문자 단위로 분할함
from langchain_text_splitters import CharacterTextSplitter

# 최대 chunk size를 500으로 지정
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=0
)
documents = text_splitter.split_documents(docs)

In [54]:
# 270번째 페이지의 문자들을 줄 바꿈을 기준으로 출력한 것
documents[10].page_content.split("\n")

['11Example 7.34 –Solution (2 of 8)',
 'The singular values of A are',
 'To find U, we compute']

# Embedding

In [41]:
from langchain_openai.embeddings import OpenAIEmbeddings
# FAISS는 벡터 저장소, 고성능 벡터 검색 라이브러리
from langchain_community.vectorstores import FAISS

embed_model = OpenAIEmbeddings(api_key="sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV",
                                model='text-embedding-3-small')

In [ ]:
emb = embed_model.embed_query("What is Mistral AI?")
# 딱히 의미나 기준이 있진 않음
emb[:3]

[-0.010324695814672812, 0.023181728425434665, 0.006363398698291552]

In [38]:
emb_doc = embed_model.embed_documents(['Hi', 'Mistal','ML'])
len(emb_doc)

2

# VectorStore

### Chroma

In [55]:
# chroma 벡터 저장소 임포트
from langchain_community.vectorstores import Chroma

# 주어진 문서들을 임베딩하고 Chroma 데이터베이스에 저장한다.
# 생성된 Chroma 데이터베이스 객체를 db변수에 할당한다.
db = Chroma.from_documents(documents, embed_model)

In [60]:
vector_index = FAISS.from_documents(documents, embed_model)
#  사용자가 질문한 문장과의 유사도를 측정함
retrieved = vector_index.similarity_search("I love Tokyo")
# 가장 유사한 문서 출력
retrieved[0].page_content

'20The Singular Value Decomposition (5 of 5)\nTheorem 7.16 \nLet A be an m ×n matrix with rank r. Then the image of the \nunit sphere in\n under the matrix transformation that maps \nx to Axis \na. the surface of an ellipsoid in\n if r = n.\nb. a solid ellipsoid in\n if r < n.'

### FAISS

In [ ]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embed_model)

In [62]:
retrieved = vector_index.similarity_search("What is MistralAI?")
retrieved[0].page_content

'23Example 7.32\nFind the pseudoinverse of\nSolution:\nFrom the definition of the pseudoinverse of A, we have'

### search_type
* mmr: maximal marginal relevance retrieval
* similarity_score_threshold
* top_k

In [64]:
query = 'OpenAI의 sora모델에 대해 알려줘'

In [65]:
# MMR 검색 방식 채택 -> 관련성과 함께 결과의 다양성도 같이 고려함
retriever = vector_index.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)

c:\Users\RSM\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='18The Singular Value Decomposition (3 of 5)\nTheorem 7.14 \nThe Outer Product Form of the SVD\nLet A be an m ×n matrix with singular values                       \nσ1≥ σ2≥ ... ≥ σr> 0 and σr+1= σr+2= ... = σn= 0. Let               \nu1, ... , urbe left singular vectors and let v1, ... , vrbe right \nsingular vectors of A corresponding to these singular values. \nThen', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 17}),
 Document(page_content='21\nThe Pseudoinverse of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 20}),
 Document(page_content='10Example 7.34 –Solution (1 of 8)\nand find that its eigenvalues are λ1= 2, λ2= 1, and λ3= 0, \nwith corresponding eigenvectors\nVerify this. These vectors are orthogonal, so we normalize \nthem to obtain', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 9}),
 Document(page_content='17Example 7

In [66]:
# 임계값 설정
retriever = vector_index.as_retriever(
    # 유사도가 0.3 이상인 문자들만 반환함
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.3}
)
retriever.get_relevant_documents(query)

c:\Users\RSM\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='18The Singular Value Decomposition (3 of 5)\nTheorem 7.14 \nThe Outer Product Form of the SVD\nLet A be an m ×n matrix with singular values                       \nσ1≥ σ2≥ ... ≥ σr> 0 and σr+1= σr+2= ... = σn= 0. Let               \nu1, ... , urbe left singular vectors and let v1, ... , vrbe right \nsingular vectors of A corresponding to these singular values. \nThen', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 17}), -0.1657379970888775), (Document(page_content='6\nThe Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 5}), -0.17365924438540903), (Document(page_content='20The Singular Value Decomposition (5 of 5)\nTheorem 7.16 \nLet A be an m ×n matrix with rank r. Then the image of the \n

[]

In [67]:
# k를 3으로 설정해서 가장 유사한 3개의 문장만 출력하도록 함
retriever = vector_index.as_retriever(search_kwargs={"k": 3})
retriever.get_relevant_documents(query)

[Document(page_content='18The Singular Value Decomposition (3 of 5)\nTheorem 7.14 \nThe Outer Product Form of the SVD\nLet A be an m ×n matrix with singular values                       \nσ1≥ σ2≥ ... ≥ σr> 0 and σr+1= σr+2= ... = σn= 0. Let               \nu1, ... , urbe left singular vectors and let v1, ... , vrbe right \nsingular vectors of A corresponding to these singular values. \nThen', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 17}),
 Document(page_content='6\nThe Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 5}),
 Document(page_content='20The Singular Value Decomposition (5 of 5)\nTheorem 7.16 \nLet A be an m ×n matrix with rank r. Then the image of the \nunit sphere in\n under the matrix transformation that maps \nx to Axis \na. the surface of an ellipsoid in\n if r = n.\nb. a solid ellipsoid in\n if r < n.', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\

### Multi Query etriever

In [68]:
# 프로그램 실행 중 발생하는 이벤트를 기록함
import logging

# 인자 없이 호출됨(기본 설정)
logging.basicConfig()
# 상세한 로깅 -> 단순한 Warning 보다 더 상세한 정보를 제공
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [69]:
# OpenAI 챗봇 모델을 Langchain 환경에서 사용할 수 있게 해줌
from langchain_openai import ChatOpenAI
# temperature = 0으로 설정하여 최대한 객관적인 답변을 받음 
llm = ChatOpenAI(temperature=0)

In [70]:
# 다수의 query를 처리할 수 있게 해주는 라이브러리
from langchain.retrievers.multi_query import MultiQueryRetriever
retriever_multi = MultiQueryRetriever.from_llm(
    retriever=vector_index.as_retriever(), llm=llm
)

In [72]:
retriever_multi.get_relevant_documents(query="Pseudoinverse에 대해 알려줘")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Pseudoinverse란 무엇인가요?', '2. Pseudoinverse에 대한 정보를 알려주세요.', '3. Pseudoinverse의 개념과 사용법을 설명해주세요.']


[Document(page_content='21\nThe Pseudoinverse of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 20}),
 Document(page_content='22The Pseudoinverse of a Matrix (1 of 2)\nDefinition\nIf A is a matrix with linearly independent columns, then the \npseudoinverse of A is the matrix\n defined by', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 21}),
 Document(page_content='24The Pseudoinverse of a Matrix (2 of 2)\nTheorem 7.12\nLet A be a matrix with linearly independent columns. Then \nthe pseudoinverse\n of A satisfies the following properties, \ncalled the Penrose conditions for A:', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 23}),
 Document(page_content='23Example 7.32\nFind the pseudoinverse of\nSolution:\nFrom the definition of the pseudoinverse of A, we have', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 22})]

### Parent Document Retriever

In [73]:
from langchain.retrievers import ParentDocumentRetriever
# 문서를 메모리에 저장하기 위한 저장소 생성
from langchain.storage import InMemoryStore

store = InMemoryStore()

# 계층적 분할
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# 컬렉션 이름 설정
vector_index = Chroma(collection_name='split_parents', embedding_function=embed_model)

retriever = ParentDocumentRetriever(
    vectorstore=vector_index,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [74]:
retriever.docstore

In [75]:
retriever.get_relevant_documents(query="Pseudoinverse에 대해 알려줘")

[]

In [76]:
retriever.add_documents(docs)

In [79]:
retriever.get_relevant_documents(query="Pseudoinverse에 대해 알려줘")

[Document(page_content='21\nThe Pseudoinverse of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 20}),
 Document(page_content='22The Pseudoinverse of a Matrix (1 of 2)\nDefinition\nIf A is a matrix with linearly independent columns, then the \npseudoinverse of A is the matrix\n defined by', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 21}),
 Document(page_content='24The Pseudoinverse of a Matrix (2 of 2)\nTheorem 7.12\nLet A be a matrix with linearly independent columns. Then \nthe pseudoinverse\n of A satisfies the following properties, \ncalled the Penrose conditions for A:', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 23}),
 Document(page_content='23Example 7.32\nFind the pseudoinverse of\nSolution:\nFrom the definition of the pseudoinverse of A, we have', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 22})]

# Generator

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [81]:
# invoke를 이용하여 모델에게 프롬프트를 전달하고 응답을 생성하도록 요청함
llm.invoke("내일 점심에 뭐 먹을까?")

AIMessage(content='점심에 먹을 음식을 선택하는 것은 개인의 취향과 식사 습관에 따라 다를 수 있습니다. 몇 가지 옵션을 드려보겠습니다.\n\n1. 한식: 비빔밥, 불고기, 된장찌개\n2. 중식: 짜장면, 탕수육, 볶음밥\n3. 양식: 파스타, 스테이크, 샐러드\n4. 일식: 초밥, 덮밥, 라멘\n5. 패스트푸드: 햄버거, 피자, 샌드위치\n\n어떤 종류의 음식을 좋아하시나요? 원하는 음식을 선택하시고 즐거운 식사 되시길 바랍니다!', response_metadata={'token_usage': {'completion_tokens': 224, 'prompt_tokens': 23, 'total_tokens': 247}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1373abd0-2647-4052-b71d-43bb27795ce9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 224, 'total_tokens': 247})

# Chain

#### simplest chain

In [82]:
# 대화형 프롬프트를 생성하는 데에 사용
from langchain_core.prompts import ChatPromptTemplate
# 체인에서 데이터를 그대로 전달하는 데에 사용
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template("Tell me something")
# 프롬프트와 언어 모델을 '|'를 통해 연결함
chain = prompt | llm

In [83]:
# 이미 chain에 prompt가 들어가 있으므로 {}로 아무것도 전달하지 않음
chain.invoke({})

AIMessage(content="Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. Honey's low water content and acidic pH create an inhospitable environment for bacteria and microorganisms, allowing it to last indefinitely.", response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 10, 'total_tokens': 71}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a6f7f7d-e36c-434b-bc08-955a296bca27-0', usage_metadata={'input_tokens': 10, 'output_tokens': 61, 'total_tokens': 71})

#### w. output parser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# 모델의 답변을 문자열로 파싱하게 도와줌
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me something")
chain = prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({})

"Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. Honey's high sugar content and low moisture levels create an environment that is inhospitable to bacteria and microorganisms, allowing it to last indefinitely."

#### w.prompting

In [85]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 주제만 나중에 따로 지정할 수 있음
prompt = ChatPromptTemplate.from_template("Tell me something about {topic}")
chain = prompt | llm | StrOutputParser()

In [86]:
chain.invoke({'topic': 'christiano ronaldo'})

"Cristiano Ronaldo is a Portuguese professional footballer who is widely regarded as one of the greatest footballers of all time. He has won numerous awards and accolades throughout his career, including multiple FIFA Ballon d'Or awards for the best player in the world. Ronaldo has played for top clubs such as Sporting Lisbon, Manchester United, Real Madrid, and Juventus, and has achieved great success at each of these clubs. He is known for his incredible goal-scoring ability, athleticism, and work ethic, and is considered one of the most complete players in the history of the sport. Ronaldo is also known for his philanthropy and charitable work, and has donated millions of dollars to various causes over the years."

#### equivalent to...

In [90]:
prompt_formatted = prompt.invoke({'topic': 'sora, developed by OpenAI'})
prompt_formatted

ChatPromptValue(messages=[HumanMessage(content='Tell me something about sora, developed by OpenAI')])

In [91]:
model_output = llm.invoke(prompt_formatted)
model_output

AIMessage(content='Sora is a large-scale language model developed by OpenAI that is designed to generate human-like text based on the input it receives. It is trained on a diverse range of internet text data to improve its ability to understand and generate natural language. Sora is capable of generating coherent and contextually relevant responses to a wide variety of prompts, making it a powerful tool for tasks such as text generation, language translation, and content creation.', response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 18, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-66ea77f5-e975-4f18-9701-545af59e0480-0', usage_metadata={'input_tokens': 18, 'output_tokens': 87, 'total_tokens': 105})

In [92]:
# 문자열만 나오도록 파싱
parser = StrOutputParser()
parser.invoke(model_output)

'Sora is a large-scale language model developed by OpenAI that is designed to generate human-like text based on the input it receives. It is trained on a diverse range of internet text data to improve its ability to understand and generate natural language. Sora is capable of generating coherent and contextually relevant responses to a wide variety of prompts, making it a powerful tool for tasks such as text generation, language translation, and content creation.'

#### Also equivalent to...

In [ ]:
# 이런 형식도 있다는 것을 알아두자
(prompt | llm | parser).invoke({'topic': 'sora, developed by OpenAI'})

'Sora is a large-scale language model developed by OpenAI that is designed to generate human-like text based on the input it receives. It is trained on a diverse range of internet text data to improve its ability to understand and generate natural language. Sora is capable of generating coherent and contextually relevant responses to a wide variety of prompts, making it a powerful tool for tasks such as text generation, language translation, and content creation.'

## RAG chain

In [87]:
# 가지고 있는 문서에서 query에 해당하는 내용을 찾아줌
retrieved_docs = retriever.invoke("Tell me something about matrix A")
retrieved_docs

[Document(page_content='22The Pseudoinverse of a Matrix (1 of 2)\nDefinition\nIf A is a matrix with linearly independent columns, then the \npseudoinverse of A is the matrix\n defined by', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 21}),
 Document(page_content='21\nThe Pseudoinverse of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 20}),
 Document(page_content='3\nThe Singular Values of a Matrix', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 2}),
 Document(page_content='24The Pseudoinverse of a Matrix (2 of 2)\nTheorem 7.12\nLet A be a matrix with linearly independent columns. Then \nthe pseudoinverse\n of A satisfies the following properties, \ncalled the Penrose conditions for A:', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 23})]

In [93]:
def merge_docs(retrieved_docs):
    # 리스트의 각 요소를 "\n\n"을 구분자로 하여 합침
    # d.page_content를 통해 문자열만 합침
    return "\n\n".join([d.page_content for d in retrieved_docs])

In [94]:
print(merge_docs(retrieved_docs))

22The Pseudoinverse of a Matrix (1 of 2)
Definition
If A is a matrix with linearly independent columns, then the 
pseudoinverse of A is the matrix
 defined by

21
The Pseudoinverse of a Matrix

3
The Singular Values of a Matrix

24The Pseudoinverse of a Matrix (2 of 2)
Theorem 7.12
Let A be a matrix with linearly independent columns. Then 
the pseudoinverse
 of A satisfies the following properties, 
called the Penrose conditions for A:


In [96]:
(retriever | merge_docs | llm | parser).invoke("Tell me something about pseudoinverse")

'\n\nA^+ = (A^T * A)^-1 * A^T\n\nGiven matrix A:\n\nA = [1 2\n     3 4\n     5 6]\n\nCalculating A^T * A:\n\nA^T * A = [1 3 5\n           2 4 6] * [1 2\n                       3 4\n                       5 6]\n        = [35 44\n           44 56]\n\nCalculating the inverse of A^T * A:\n\n(A^T * A)^-1 = 1/(35*56 - 44*44) * [56 -44\n                                   -44 35]\n            = 1/4 * [56 -44\n                     -44 35]\n            = [14 -11\n               -11 8]\n\nCalculating A^+:\n\nA^+ = (A^T * A)^-1 * A^T\n    = [14 -11\n       -11 8] * [1 3 5\n                 2 4 6]\n    = [14*1 - 11*2 14*3 - 11*4 14*5 - 11*6\n       -11*1 + 8*2 -11*3 + 8*4 -11*5 + 8*6]\n    = [-8 -2 4\n       6 2 -2]\n\nTherefore, the pseudoinverse of A is:\n\nA^+ = [-8 -2\n        6 2\n        4 -2]'

### Runnable Parallel

#### https://python.langchain.com/docs/expression_language/how_to/map

In [97]:
# 병렬 실행 체인을 구성할 수 있도록 도와줌
from langchain_core.runnables import RunnableParallel

# retruever를 사용하여 관련 문서를 검색함과 동시에
# llm을 사용하여 언어 모델 작업을 수행함
chain_parallel = RunnableParallel({"context": retriever, "llm": llm})
chain_parallel

{
  context: ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000015095C44E60>, docstore=<langchain_core.stores.InMemoryStore object at 0x0000015095012C60>, child_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x0000015095C46CC0>, parent_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x0000015095C448F0>),
  llm: ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000150950104D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000015095C44EC0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
}

In [98]:
chain_parallel.invoke("Tell me something about sora, developed by OpenAI")

{'context': [Document(page_content='6\nThe Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 5}),
  Document(page_content='Copyright ©  Cengage Learning. All rights reserved. \n7.4The Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 1}),
  Document(page_content='20The Singular Value Decomposition (5 of 5)\nTheorem 7.16 \nLet A be an m ×n matrix with rank r. Then the image of the \nunit sphere in\n under the matrix transformation that maps \nx to Axis \na. the surface of an ellipsoid in\n if r = n.\nb. a solid ellipsoid in\n if r < n.', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 19}),
  Document(page_content='17Example 7.34 –Solution (8 of 8)\nand we have the SVD', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 16})],
 'llm': AIMessage(content='Sora is a large-scale lan

#### RunnablePassthrough(병렬 처리)

In [99]:
# 입력을 그대로 통과시키는 역할을 함
from langchain_core.runnables import RunnablePassthrough

# 원본 query를 그대로 전달
chain_parallel = RunnableParallel({"context": retriever, "query": RunnablePassthrough()})
chain_parallel

{
  context: ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000015095C44E60>, docstore=<langchain_core.stores.InMemoryStore object at 0x0000015095012C60>, child_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x0000015095C46CC0>, parent_splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x0000015095C448F0>),
  query: RunnablePassthrough()
}

In [100]:
chain_parallel.invoke("Tell me something about sora, developed by OpenAI")

{'context': [Document(page_content='6\nThe Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 5}),
  Document(page_content='Copyright ©  Cengage Learning. All rights reserved. \n7.4The Singular Value Decomposition', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 1}),
  Document(page_content='20The Singular Value Decomposition (5 of 5)\nTheorem 7.16 \nLet A be an m ×n matrix with rank r. Then the image of the \nunit sphere in\n under the matrix transformation that maps \nx to Axis \na. the surface of an ellipsoid in\n if r = n.\nb. a solid ellipsoid in\n if r < n.', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 19}),
  Document(page_content='17Example 7.34 –Solution (8 of 8)\nand we have the SVD', metadata={'source': 'C:\\Users\\RSM\\OneDrive\\바탕 화면\\API 스터디\\Chapter 7.pdf', 'page': 16})],
 'query': 'Tell me something about sora, developed b

In [101]:
template = """
Utilizing the context given below, answer the question.
[context]
{context}

question: {query}
"""

prompt = ChatPromptTemplate.from_template(template)
# query를 그대로 전달함
# 파싱하여 문자열만 출력함
chain = RunnableParallel({"context": retriever, "query": RunnablePassthrough()})\
        | prompt \
        | llm \
        | StrOutputParser()

In [102]:
chain.invoke("Tell me something about pseudoinverse od matrix A")

'The pseudoinverse of matrix A is defined as the matrix that is obtained when A has linearly independent columns. It satisfies certain properties known as the Penrose conditions for A.'

#### equivalent to...

In [ ]:
chain = RunnableParallel(context=retriever, query=RunnablePassthrough()) \
        | prompt \
        | llm \
        | StrOutputParser()

In [ ]:
chain.invoke("Tell me something about sora, developed by OpenAI")

'Sora is a text-to-video model developed by OpenAI that can generate videos based on short descriptive prompts and extend existing videos forwards or backwards in time. It can generate videos with resolutions up to 1920x1080 or 1080x1920. The team named it after the Japanese word for sky to signify its "limitless creative potential". Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model. OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for the purpose. The technology was demonstrated to the public on February 15, 2024, showcasing the ability to generate videos up to one minute long.'

### Anything can be passed

In [ ]:
# 항상 bar를 반환하는 함수
def foo(anything):
    return 'bar'

In [ ]:
# 세 가지 일을 동시에 수행하는 병렬 체인
# bar에 원하는 추가정보를 넣음으로써 구체적으로 원하는 결과를 얻을 수 있다.
chain = RunnableParallel(context=retriever, query=RunnablePassthrough(), asdf=foo) \
        | prompt \
        | llm \
        | StrOutputParser()

In [ ]:
chain.invoke("Tell me something about sora, developed by OpenAI")

'Sora is a text-to-video model developed by OpenAI that can generate videos based on short descriptive prompts and extend existing videos forwards or backwards in time. It can generate videos with resolutions up to 1920x1080 or 1080x1920. The team named Sora after the Japanese word for sky to signify its "limitless creative potential". The technology behind Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model. OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for the purpose. Sora was demonstrated to the general public on February 15, 2024, with the technology being able to generate videos up to one minute long.'

#### also equivalent to...

In [ ]:
chain = {"context": retriever, "query": RunnablePassthrough()} \
        | prompt \
        | llm \
        | StrOutputParser()

In [ ]:
chain.invoke("Tell me something about sora, developed by OpenAI")

'Sora is a text-to-video model developed by OpenAI that can generate videos based on short descriptive prompts and extend existing videos forwards or backwards in time. It can generate videos with resolutions up to 1920x1080 or 1080x1920. The team named it after the Japanese word for sky to signify its "limitless creative potential". Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model. OpenAI demonstrated Sora\'s capabilities by showcasing high-definition videos to the public, stating that the technology was able to generate videos up to one minute long.'

#### Add formatting

In [ ]:
chain = {"context": retriever, "query": RunnablePassthrough()} \
        | prompt

# 생성된 첫 번째 메시지의 내용을 출력함
print(chain.invoke("Tell me something about sora, developed by OpenAI").messages[0].content)


Utilizing the context given below, answer the question.
[context]
[Document(page_content='=== Text\n\nto\n\nvideo ===\n\n==== Sora ====\n\nSora is a text-to-video model that can generate videos based on short descriptive prompts as well as extend existing videos forwards or backwards in time. It can generate videos with resolution up to 1920x1080 or 1080x1920. The maximal length of generated videos is unknown. The team that developed Sora named it after the Japanese word for sky to signify its "limitless creative potential". The technology behind Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model. OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for the purpose, but did not reveal the number or the exact source of the videos.OpenAI demonstrated a few Sora-created high-definition videos to the general public on February 15, 2024, stating that the technology was able to generate videos up to one minute long. It

In [103]:
chain = {"context": retriever | merge_docs, "query": RunnablePassthrough()} \
        | prompt

print(chain.invoke("Tell me something about pseudoinverse od matrix A ").messages[0].content)


Utilizing the context given below, answer the question.
[context]
22The Pseudoinverse of a Matrix (1 of 2)
Definition
If A is a matrix with linearly independent columns, then the 
pseudoinverse of A is the matrix
 defined by

21
The Pseudoinverse of a Matrix

24The Pseudoinverse of a Matrix (2 of 2)
Theorem 7.12
Let A be a matrix with linearly independent columns. Then 
the pseudoinverse
 of A satisfies the following properties, 
called the Penrose conditions for A:

23Example 7.32
Find the pseudoinverse of
Solution:
From the definition of the pseudoinverse of A, we have

question: Tell me something about pseudoinverse od matrix A 



### Utilizing prompt

In [105]:
# 템플릿, 체인, 에이전트 등을 공유하고 관리할 수 있는 hub
from langchain import hub


In [109]:
# hub의 특정 프롬프트 템플릿을 가져옴
prompt = hub.pull("rlm/rag-prompt")
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]

In [ ]:
# hub에 있는 템플릿 사용
chain = RunnableParallel(context=retriever, question=RunnablePassthrough()) \
        | prompt \
        | llm \
        | StrOutputParser()

In [ ]:
chain.invoke("Tell me something about sora, developed by OpenAI")

'Sora is a text-to-video model developed by OpenAI that can generate videos based on short descriptive prompts and extend existing videos in time. It can create videos with resolutions up to 1920x1080 or 1080x1920 and was named after the Japanese word for sky to symbolize its limitless creative potential. OpenAI trained Sora using publicly available and licensed videos, demonstrating its capabilities to generate videos up to one minute long.'

# Putting it all together

In [110]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [120]:
loader = DirectoryLoader(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디", glob="*", show_progress=True)

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

embed_model = OpenAIEmbeddings(api_key="sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV",
                                model='text-embedding-3-small')

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [121]:
# 리스트 요소들 합치는 함수
def merge_docs(retrieved_docs):
    return "\n\n".join([d.page_content for d in retrieved_docs])

In [122]:
docs = loader.load()
documents = text_splitter.split_documents(docs)
# 문서 안의 데이터를 벡터 임베딩함
vector_index = FAISS.from_documents(documents, embed_model)
# mmr 검색 방식 채택
retriever = vector_index.as_retriever(search_type="mmr")




ERROR:langchain_community.document_loaders.directory:Error loading file C:\Users\RSM\OneDrive\바탕 화면\API 스터디\Chapter 7.pdf


ImportError: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"

In [123]:
chain = RunnableParallel(context=retriever, question=RunnablePassthrough()) \
        | prompt \
        | llm \
        | StrOutputParser()

In [125]:
chain.invoke("Tell me something about pseudoinverse of matrix")

'The pseudoinverse of a matrix is defined when the matrix has linearly independent columns. It satisfies certain properties known as the Penrose conditions. It is used in cases where the matrix does not have an inverse.'